# OCR 

## 개요

* OCR = Text detection + Text recognition
* Text detection : 문자의 영역을 검출
* Text recognition : 검출된 영역의 문자를 인식

실습 목표
* Deep learning 기반의 OCR을 이해합니다.
* Text를 Detection하기 위한 딥러닝 기법을 배웁니다.
* Text를 Recognize하기 위한 딥러닝 기법을 배웁니다

## Before Deep Learning

* OCR = Optical Character Recognition
* Deep learning이 사용되기 이전에는 Tesseract OCR을 사용하였음

## Text detection

* 이미지 내에서 문자를 검출해낼 때엔 검출하기 위한 최소단위를 지정해야함.\
* end-to-end : 입력에서 출력까지 파이프라인 네트워크 없이 신경망으로 한번에 처리
* Aspect ratio(종횡비) : 가로와 세로의 비, 논문에서는 하나의 숫자로 표현
* offset : 특정한 값에서 차이가 나는 값 또는 차이. 오차랑은 다른 것

### 1. Regression

* TextBoxes : Deep learning 기반의 Detection을 이용하여 단어 단위로 인식.
* 네트워크의 기본 구조는 SSD:single shot multibox detector를 활용

* 일반적으로 단어는 가로로 길기 때문에 Aspect ratio(종횡비)가 큽니다.
* 이에 따라 기존 SSD에서 Regression을 위한 Convolution layer의 kernel의 크기가 3x3에서 1x5로 정의되어 사용합니다.
* Anchor box의 aspect ratio를 1, 2, 3, 5, 7로 만들고, 이에 vertical offset을 적용하여 세로 방향으로 촘촘한 단어의 배열에 대응하도록 했음

### 2. Segmentation

* 글자와 배경을 분리
* 문자들이 매우 촘촘하니 글자 영역으로 찾아낸 뒤에 이를 분리해내는 작업이나 연결하는 작업을 더 해서 원하는 최소단위로 만들어줘야함.

* PixelLink는 Text 영역을 찾아내는 segmentation과 함께, 글자가 어느 방향으로 연결되는지를 같이 학습하여
* Text 영역 간의 분리 및 연결을 할 수 있는 정보를 추가적으로 활용
* PixelLink의 output은 총 9가지 정보(Text/non-text Prediction을 위한 class segmentation map(1개), Link Prediction map(8개))
* Text/non-text prediction : 해당 영역이 Text 인지 Non-text인지 예측 값을 의미하는 2개의 커널
* Link Prediction map : 글자의 pixel을 중심으로 인접한 8개의 Pixel에 대한 연결 여부를 의미하는 16개의 커널로 이루어짐
* conv 1x1, 2(16)의 형태가 U-Net 구조로 연결, 인접 pixel간 연결 구조가 지속적으로 유지되게 하는 모델 구조
* 이를 통해 인접한 pixel이 중심 pixel과 단어 단위로 연결된 pixel인지, 분리된 pixel인지 알 수 있슴.
* 이로써 문자 영역이 단어 단위로 분리된 Instance segmentation이 가능

### 3. Recent studies

#### 3.1 CRAFT

* 문자 단위로 문자의 위치를 찾아낸 뒤, 이를 연결하는 방식은 Segmentation 기반으로 구현하는 방법
* 문자의 영역을 boundary로 명확하게 구분하지 않고, 가우시안 분포를 따르는 원형의 score map을 만들어서 배치시키는 방법으로 문자의 영역을 학습
* 문자 단위 라벨을 가진 데이터셋이 많지 않기 때문에, 단어 단위의 정보만 있는 데이터셋에 대해 단어의 영역에 Inference를 한 후
* 얻어진 문자 단위의 위치를 다시 학습에 활용하는 Weakly supervised learning을 활용

#### 3.2 Pyramid Mask Text Detector

* Mask-RCNN의 구조를 활용하여 Text 영역을 Region proposal network로 찾아냅니다.
* Box head를 더 정확하게 regression 및 classification을 하고 Mask head에서 Instance의 Segmentation하는 과정을 거침.
* Mask 정보가 부정확한 경우를 반영하기 위해 Soft-segmentation을 활용합니다.
* 단어 영역이 Box Head에 의해 빨간색으로 잡히면 이미지의 baseline처럼 boundary 모두 Text 영역으로 잡지만,
* PMTD는 단어의 사각형 배치 특성을 반영하여 피라미드 형태의 Score map을 활용합니다.
* 따라서 Pyramid 형상의 Mask를 갖게 되어 Pyramid Mask Text detector란 이름이 생겼습니다.

## Text recognition

### 1. Unsegmented Data

* 글자 이미지는 문자의 순서대로 정보를 가짐. "YOU"라는 이미지에서 "Y", "O", "U"의 영역은 이미지상에 분리가 가능
* 분리에 드는 비용이 많이 들거나 어려워 Segmentation 되어있지 않은 데이터를 Unsegmented data

### 2. CNN + RNN = CRNN

* Unsegmented Data : Segment 되어 있지 않은 하위 데이터들끼리 시퀀스를 이루고 있다.
* CNN와 RNN을 같이 쓰자 = CRNN
* Feature Extractor : 문자 이미지와 정보를 추출 - CNN 기반의 VGG 또는 ResNet과 같은 네트워크 사용
* 추출한 Feature를 Map-To-Sequence를 통해 Sequence 형태의 featur로 변환 후 다양한 길이의 Input을 처리할 수 있는 RNN으로 넣음

* RNN이 Feature로부터 문자를 인식하기 위해서는 문자 영역처럼 넓은 정보가 필요 - LSTM으로 구성
* 앞의 정보뿐 아니라 뒤의 정보도 필요하기 때문에 이를 Bidirectional로 구성해 Bidirectional LSTM을 사용.
* Bidirectional LSTM을 사용해 step마다 나오는 결과를 Transcription Layer에서 문자로 변환 

### 3. CTC

* CRNN에서 Step마다 Fully Connected Layer의 logit을 Softmax 함수에 넣어줌으로써 어떤 문자일 확률이 높은지 알 수 있습니다.
* 이 결과 그대로 문자로 변환하면 기대한 것과 다른 결과가 나옵니다.
* 모델의 Output은 24개의 글자로 이루어진 Sequence이지만, 실제 결과와 이는 다르기 때문
* HELLO 라는 이미지가 들어오면 이것의 Output이 HHHEEELLLOOOO 와 같이 24자의 sequence를 보게 됩니다.
* 24자의 Sequence를 실제 인식 결과로 바꾸기 위해서는 어떤 방법을 써야 할까요?

* CRNN에서 Unsegmented Data를 위해 CTC(Connectionist Temporal Classification)를 활용.
* CTC는 Unsegmented data와 같이 입력과 출력이 서로 다른 길이의 시퀀스를 가질 때, 이를 조정 없이 활용하는 방법
* CTC의 핵심인 모델의 Output에서 Label sequence의 확률을 구할 수 있는 방법에 대해 알아보겠습니다.

* 위에서 본 HHHEEELLLOOOO를 HELLO로 만들기 위해 중복되는 단어를 하나로 하여 HELO로 바꿀 수 있습니다.
* 여기서 반복되는 LL은 어떻게 구분할 까요?
* Label encode에서 이렇게 같은 문자를 구분하기 위해 Blank를 중복된 라벨 사이에 구분하기 위해 넣어줍니다.

### TPS

* OCR이 어려워지는 이유 : 불규칙한 방향이나 휘어진 진행 방향 때문
* Thin Plate Spline (TPS) Transformation을 적용하여 입력 이미지를 단어 영역에 맞게 변형 시켜 인식이 잘 되도록 해줌
* control point를 정의, 해당 point들이 특정 위치로 옮겨졌을 때, 축 방향의 변화를 interpolation하여 모든 위치의 변화 추정.
* 이를 통해 전체 이미지의 pixel의 변화를 control point로 만들어낼 수 있음
* Spatial Transformer Network를 통해서 Control point가 얼마나 움직여야 하는지 예측하는 네트워크를 Recognition model 앞단에 붙여 입력 이미지를 정방향으로 맞춰줍니다.
* TPS 연산은 미분 가능한 연산이기 때문에 이 모듈을 Recognition model 앞단에 붙여서 학습이 바로 가능합니다.

## Text recognition + Attention

### Attention sequence prediction

* CTC를 활용한 CRNN의 경우, column에 따라서 prediction된 Label의 중복된 것들을 제거해 줌으로써 원하는 형태의 label로 만들기
* Attention 기반의 sequence prediction은 문장 길이를 고정하고, 입력되는 Feature에 대한 Attention을 기반으로 해당 글자의 Label을 prediction 합니다.
* RNN으로 Character label을 뽑아낸다고 생각하면 되는데, 첫 번째 글자에서 입력 feature에 대한 Attention을 기반으로 label을 추정하고, 추정된 label을 다시 입력으로 사용하여 다음 글자를 추정해내는 방식
* 미리 정해둔 Token을 사용해 'start' token 과 'end' token이 있습니다.
* 필요에 따라서 예외처리나 공백을 위한 token을 만들어서 사용하기도 합니다.

###  Transformer와 함께

* 불규칙한 텍스트를 잘 인식하기 위해 2D 공간에 대한 attention을 활용하여 문자를 인식하기 위해 Transformer를 활용
* Attention의 핵심은 Decoder의 현재 포지션에서 중요한 Encoder의 State에 가중치가 높게 매겨진다는 점
* Attention이 시각화된 이미지를 눈으로 볼 수 있는데요, Decoder의 각 Step에 따라 입력에 대한 Visual Attention이 시각화